In [1]:
import os
import sys
home_dir = "../../"
module_path = os.path.abspath(os.path.join(home_dir))
if module_path not in sys.path:
    sys.path.append(module_path)

import random
import numpy as np
import pandas as pd
from models.aa_common.performance_metrics import *

In [15]:
method_names = ['random_classifier', 'sift', 'polyphen2_HVAR', 'metarnn', 'revel', 'mvp', 'sequnet', 
           'cadd_raw', 'integrated_fitCons', 'phyloP17way_primate', 'phastCons17way_primate', 'bStatistic', 'conservation',
           'esm1b_t33_650M_UR50S', 'esm1v_t33_650M_UR90S', 'esm2_t33_650M_UR50D', 
           'prottrans_bert_bfd', 'prottrans_t5_xl_u50', 'vespa', 'proteinbert', 'protbert', 'unirep']

# by default, we consider larger means positive class (here Effect/Knock-out), but the following models have specific opposite meaning
methods_smaller_means_damaging = ["sequnet", "sift"]
metrics = ["AUC-ROC", "AUC-PR", "F1-max", "Th-max", "Precision", "Recall", "Accuracy", "Balanced-accuracy", "MCC"]

In [18]:
def sample_positive_and_negative_data_points(df, method_name, positive_cls, negative_cls, n_samples):
    df = df[(df["class"]==positive_cls) | (df["class"]==negative_cls)]
    df = df[~pd.isna(df[method_name])]  # taking only non-NAN values

    positive_cls_df = df[df["class"]==positive_cls]
    negative_cls_df = df[df["class"]==negative_cls]
    
    positive_cls_df["class_numeric"] = 1
    negative_cls_df["class_numeric"] = 0

    positive_cls_df = positive_cls_df.sample(n=min(n_samples, positive_cls_df.shape[0]))
    negative_cls_df = negative_cls_df.sample(n=min(n_samples, negative_cls_df.shape[0]))

    sampled_df = pd.concat([positive_cls_df, negative_cls_df])
    return sampled_df

def calibrate_prediction_scores_direction(df, method_name):
    if method_name in methods_smaller_means_damaging:
        df['pred'] = df['pred'].multiply(-1)
    
    auc_roc_score, larger_means_positive_class = get_auc_roc_score(df)
    if not larger_means_positive_class:
        df['pred'] = df['pred'].multiply(-1)
    return df, auc_roc_score

def compute_performance_metrics(df, method_name, positive_cls, negative_cls, n_samples):
    metric_scores = []
    for i_run in range(10): #10 runs
        if method_name=="random_classifier": df[method_name] = [random.uniform(0, 1) for i in range(df.shape[0])]

        sampled_df = sample_positive_and_negative_data_points(df, method_name, positive_cls, negative_cls, n_samples)
        sampled_df["pred"]=(sampled_df[method_name]-sampled_df[method_name].min())/(sampled_df[method_name].max()-sampled_df[method_name].min()) # scaling prediction scores between [0, 1]
        
        sampled_df, auc_roc_score =  calibrate_prediction_scores_direction(sampled_df, method_name)
        
        # ks_statistic, ks_pvalue = get_KS_test_score(sampled_df)
        auc_pr_score, precisions, recalls, thresholds = get_auc_pr_score(sampled_df)
        f1_max, th_max, precisions, recalls, thresholds = get_f1max_and_th(precisions, recalls, thresholds)
        precision = get_precision_score(sampled_df, th_max)
        recall = get_recall_score(sampled_df, th_max)
        accuracy = get_accuracy_score(sampled_df, th_max)
        balanced_accuracy = get_balanced_accuracy_score(sampled_df, th_max)
        mcc = get_matthews_corrcoef(sampled_df, th_max)
        
        metric_scores.append([auc_roc_score, auc_pr_score, f1_max, th_max, precision, recall, accuracy, balanced_accuracy, mcc])
    return metric_scores


def write_metrics_outputs(performance_scores_dict, output_file):
    out = open(output_file, 'w')
    out.write("Models\\Metrics")
    for metric in metrics:
        out.write(f"\t{metric}")
    out.write("\n")

    for (model_name, performance_scores) in performance_scores_dict.items():
        out.write(f"{model_name}")
        for scores in performance_scores:
            for score in scores:
                out.write(f"\t{score:.3f}")
            out.write("\n")
        out.write("\n")
        
    for (model_name, performance_scores) in performance_scores_dict.items():
        out.write(f"{model_name}")    
        avg_scores = np.mean(performance_scores, axis=0)
        std_scores = np.std(performance_scores, axis=0)
        for i in range(len(avg_scores)):
            out.write(f"\t{avg_scores[i]:.3f}({std_scores[i]:.3f})")
        out.write("\n")
    out.close()

In [21]:
from data_loader import get_merged_scores_df
task = "pmd"
result_df = get_merged_scores_df(task, home_dir)

# Effect, Knock-out
positive_cls, negative_cls, n_samples = "Knock-out", "No-effect", 1000

performance_scores_dict = {}
for method_name in method_names:
    performance_scores_dict[method_name] = compute_performance_metrics(result_df, method_name, positive_cls, negative_cls, n_samples)
    # break
write_metrics_outputs(performance_scores_dict, output_file=home_dir+f"models/aa_common/performance_analysis/{task}_{positive_cls}_vs_{negative_cls}.tsv")

Index(['mut_id', 'md5', 'pmd_id', 'nr', 'prot_acc_version', 'snp_id',
       'mut_real', 'wt', 'mut', 'prot_pos', 'chrom', 'chrom_pos', 'ref_allele',
       'alt_allele', 'function', 'source', 'crossref', 'function_summarized',
       'class', 'sift', 'polyphen2_HVAR', 'metarnn', 'revel', 'mvp',
       'cadd_raw', 'integrated_fitCons', 'phyloP17way_primate',
       'phastCons17way_primate', 'bStatistic', 'esm1b_t33_650M_UR50S',
       'esm1v_t33_650M_UR90S', 'esm2_t33_650M_UR50D', 'prottrans_bert_bfd',
       'prottrans_t5_xl_u50', 'vespa', 'proteinbert', 'sequnet', 'protbert',
       'unirep', 'conservation'],
      dtype='object')
(7179, 40)
Effect       3818
No-effect    1777
Knock-out    1584
Name: class, dtype: int64
	AUC-ROC: 0.507
	AUC-PR: 0.503
	Best F1-Score: 0.667 at threshold: 0.000
	Precision score: 0.500 at threshold: 0.000
	Recall score: 0.999 at threshold: 0.000
	Accuracy score: 0.500 at threshold: 0.000
	Balanced accuracy score: 0.500 at threshold: 0.000
	MCC score: 0.0

/tmp/ipykernel_3318003/74410669.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  positive_cls_df["class_numeric"] = 1
/tmp/ipykernel_3318003/74410669.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  negative_cls_df["class_numeric"] = 0
/tmp/ipykernel_3318003/74410669.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

	AUC-ROC: 0.495
	AUC-PR: 0.507
	Best F1-Score: 0.668 at threshold: -0.997
	Precision score: 0.502 at threshold: -0.997
	Recall score: 0.998 at threshold: -0.997
	Accuracy score: 0.503 at threshold: -0.997
	Balanced accuracy score: 0.503 at threshold: -0.997
	MCC score: 0.043 at threshold: -0.997
	AUC-ROC: 0.490
	AUC-PR: 0.513
	Best F1-Score: 0.667 at threshold: -0.999
	Precision score: 0.500 at threshold: -0.999
	Recall score: 0.999 at threshold: -0.999
	Accuracy score: 0.500 at threshold: -0.999
	Balanced accuracy score: 0.500 at threshold: -0.999
	MCC score: 0.013 at threshold: -0.999
	AUC-ROC: 0.781
	AUC-PR: 0.787
	Best F1-Score: 0.749 at threshold: -0.051
	Precision score: 0.649 at threshold: -0.051
	Recall score: 0.885 at threshold: -0.051
	Accuracy score: 0.703 at threshold: -0.051
	Balanced accuracy score: 0.703 at threshold: -0.051
	MCC score: 0.436 at threshold: -0.051
	AUC-ROC: 0.782
	AUC-PR: 0.789
	Best F1-Score: 0.741 at threshold: -0.039
	Precision score: 0.655 at threshol

/tmp/ipykernel_3318003/74410669.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  positive_cls_df["class_numeric"] = 1
/tmp/ipykernel_3318003/74410669.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  negative_cls_df["class_numeric"] = 0
/tmp/ipykernel_3318003/74410669.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

	Precision score: 0.632 at threshold: 0.327
	Recall score: 0.906 at threshold: 0.327
	Accuracy score: 0.690 at threshold: 0.327
	Balanced accuracy score: 0.690 at threshold: 0.327
	MCC score: 0.420 at threshold: 0.327
	AUC-ROC: 0.787
	AUC-PR: 0.776
	Best F1-Score: 0.749 at threshold: 0.327
	Precision score: 0.638 at threshold: 0.327
	Recall score: 0.908 at threshold: 0.327
	Accuracy score: 0.696 at threshold: 0.327
	Balanced accuracy score: 0.696 at threshold: 0.327
	MCC score: 0.433 at threshold: 0.327
	AUC-ROC: 0.801
	AUC-PR: 0.795
	Best F1-Score: 0.751 at threshold: 0.300
	Precision score: 0.637 at threshold: 0.300
	Recall score: 0.913 at threshold: 0.300
	Accuracy score: 0.697 at threshold: 0.300
	Balanced accuracy score: 0.697 at threshold: 0.300
	MCC score: 0.436 at threshold: 0.300
	AUC-ROC: 0.790
	AUC-PR: 0.785
	Best F1-Score: 0.746 at threshold: 0.319
	Precision score: 0.638 at threshold: 0.319
	Recall score: 0.899 at threshold: 0.319
	Accuracy score: 0.694 at threshold: 0.319

...

/tmp/ipykernel_3318003/74410669.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  positive_cls_df["class_numeric"] = 1
/tmp/ipykernel_3318003/74410669.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  negative_cls_df["class_numeric"] = 0
/tmp/ipykernel_3318003/74410669.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

	Accuracy score: 0.506 at threshold: 0.004
	Balanced accuracy score: 0.506 at threshold: 0.004
	MCC score: 0.078 at threshold: 0.004
	AUC-ROC: 0.560
	AUC-PR: 0.563
	Best F1-Score: 0.671 at threshold: 0.006
	Precision score: 0.507 at threshold: 0.006
	Recall score: 0.991 at threshold: 0.006
	Accuracy score: 0.513 at threshold: 0.006
	Balanced accuracy score: 0.513 at threshold: 0.006
	MCC score: 0.089 at threshold: 0.006
	AUC-ROC: 0.557
	AUC-PR: 0.553
	Best F1-Score: 0.667 at threshold: 0.003
	Precision score: 0.502 at threshold: 0.003
	Recall score: 0.996 at threshold: 0.003
	Accuracy score: 0.503 at threshold: 0.003
	Balanced accuracy score: 0.503 at threshold: 0.003
	MCC score: 0.041 at threshold: 0.003
	AUC-ROC: 0.575
	AUC-PR: 0.577
	Best F1-Score: 0.670 at threshold: 0.004
	Precision score: 0.505 at threshold: 0.004
	Recall score: 0.997 at threshold: 0.004
	Accuracy score: 0.509 at threshold: 0.004
	Balanced accuracy score: 0.509 at threshold: 0.004
	MCC score: 0.083 at threshold: 

/tmp/ipykernel_3318003/74410669.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  positive_cls_df["class_numeric"] = 1
/tmp/ipykernel_3318003/74410669.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  negative_cls_df["class_numeric"] = 0
/tmp/ipykernel_3318003/74410669.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

	Accuracy score: 0.563 at threshold: 0.006
	Balanced accuracy score: 0.563 at threshold: 0.006
	MCC score: 0.189 at threshold: 0.006
	AUC-ROC: 0.608
	AUC-PR: 0.614
	Best F1-Score: 0.681 at threshold: 0.006
	Precision score: 0.533 at threshold: 0.006
	Recall score: 0.942 at threshold: 0.006
	Accuracy score: 0.559 at threshold: 0.006
	Balanced accuracy score: 0.559 at threshold: 0.006
	MCC score: 0.184 at threshold: 0.006


/tmp/ipykernel_3318003/74410669.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  positive_cls_df["class_numeric"] = 1
/tmp/ipykernel_3318003/74410669.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  negative_cls_df["class_numeric"] = 0
